In [12]:
#Standard python libraries
import os
import time
import logging
import pickle
 
# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import torch
# Imports from our package
import lightautoml
from lightautoml.automl.base import AutoML
from lightautoml.ml_algo.boost_lgbm import BoostLGBM
from lightautoml.ml_algo.tuning.optuna import OptunaTuner
from lightautoml.pipelines.features.lgb_pipeline import LGBSimpleFeatures
from lightautoml.pipelines.ml.base import MLPipeline
from lightautoml.pipelines.selection.importance_based import ImportanceCutoffSelector, ModelBasedImportanceEstimator
from lightautoml.reader.base import PandasToPandasReader
from lightautoml.tasks import Task
#from lightautoml.utils.profiler import Profiler
from lightautoml.automl.blend import WeightedBlender
from lightautoml.dataset.roles import DatetimeRole
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.report import ReportDeco

In [27]:
df = pd.read_csv(r"C:\Users\kaush\Documents\6390_Kaggle_Project_2\new_train.csv")

In [28]:
df = df.drop(columns=["Unnamed: 0"])
df

,S2_B2_jan,S2_B3_jan,S2_B4_jan,S2_B5_jan,S2_B6_jan,S2_B7_jan,S2_B8_jan,S2_B8A_jan,S2_B9_jan,S2_B11_jan,...,S2_B11_dec,S2_B12_dec,S1_VV_dec,S1_VH_dec,ERA5_temperature_2m_dec,ERA5_total_precipitation_dec,topo_elevation_dec,topo_slope_dec,NDVI_dec,LABELS
0,-14.271277,-21.134172,957.531174,1019.557045,1230.022834,1435.138891,1693.702270,1805.411109,1891.557355,2020.567257,...,2203.025469,795.862978,2747.777894,1688.997611,289.382220,0.000147,1886.384195,0.749163,0.307887,0.0
1,-12.341429,-14.744978,949.227883,895.845779,987.517322,1350.496916,1930.239804,2048.686546,2375.680146,2259.903290,...,2922.382762,1140.796180,2528.590979,1934.261859,266.578370,0.002876,69.876216,1.385904,0.097779,1.0
2,-12.317847,-17.026201,1087.616069,1086.401035,1185.755955,1529.674085,1858.981635,1904.667487,2082.954737,2115.713139,...,2548.726966,521.102434,1556.082048,741.212901,276.467873,0.001622,91.279743,2.125908,0.418492,1.0
3,-15.332478,-20.978203,2987.224569,3163.553102,3412.572434,3908.443371,4044.232162,4231.353770,3682.709329,4518.882323,...,2537.545246,1073.588565,2388.141112,1367.682434,276.101799,0.003720,767.817294,8.427714,0.336528,1.0
4,-11.739502,-11.374262,691.825857,782.111227,478.448214,976.753128,3189.134129,3813.893119,3861.824527,4174.591554,...,4276.302643,371.861994,2080.641023,692.730434,298.663246,0.001763,61.500923,1.203899,0.595404,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59619,-8.163415,-15.237238,987.976575,1151.249683,1243.495362,1570.346203,2985.256589,3424.652420,3504.617647,3786.160943,...,2846.906155,545.811116,4877.934218,2939.084766,293.583787,0.008626,773.400823,3.750506,0.306910,0.0
59620,-11.412780,-18.545065,1961.306816,1904.491667,2287.616313,2794.906732,2868.472367,2952.698071,2972.389632,3199.591485,...,7299.094172,2067.865931,6147.303097,4271.877307,266.000834,0.003052,195.505610,1.661317,0.076089,1.0
59621,-19.757668,-18.974376,1369.005380,1696.975995,2378.074718,2426.659176,2511.191824,2731.803974,2676.593677,2916.115610,...,2695.952941,512.207308,3990.546419,2672.906513,299.309719,0.001287,513.766742,3.598575,0.274659,0.0
59622,-10.536396,-15.386933,3022.282909,3368.075760,3736.875439,4000.780072,4828.072025,5342.635968,5117.677939,5777.358267,...,4433.778527,902.604485,1769.948892,660.408789,292.725187,0.011576,496.085568,12.537847,0.627107,1.0


In [29]:
df.describe()

,S2_B2_jan,S2_B3_jan,S2_B4_jan,S2_B5_jan,S2_B6_jan,S2_B7_jan,S2_B8_jan,S2_B8A_jan,S2_B9_jan,S2_B11_jan,...,S2_B11_dec,S2_B12_dec,S1_VV_dec,S1_VH_dec,ERA5_temperature_2m_dec,ERA5_total_precipitation_dec,topo_elevation_dec,topo_slope_dec,NDVI_dec,LABELS
count,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,...,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000,59624.000000
mean,-11.795119,-18.078254,2081.046712,2100.501195,2304.266747,2667.026142,3009.954514,3153.526957,3219.598875,3318.671884,...,3354.115868,1292.793264,2070.074244,1295.342055,280.991866,0.003141,502.305559,4.679795,0.259815,0.676976
std,4.208928,4.720095,1934.856503,1933.397044,2085.701681,2339.775198,1972.864966,1919.396478,1926.329600,1891.359026,...,1894.715573,1211.689623,1166.978098,839.315313,14.017163,0.003583,586.159308,6.390935,0.196771,0.467636
min,-57.171004,-74.840663,-56.392936,299.238220,308.116962,136.928851,169.569203,84.967215,88.123196,13.675767,...,190.315943,30.882181,49.438014,47.248169,240.090890,0.000091,-119.033169,-0.163469,-0.643869,0.000000
25%,-13.911653,-20.583776,951.073058,949.720040,992.328940,1251.564736,1782.774981,1885.205452,1950.921249,2040.184965,...,2059.526734,551.741199,1276.246327,748.817994,269.644897,0.000978,124.780914,1.320973,0.100172,0.000000
50%,-11.504172,-17.643541,1163.268277,1210.112302,1437.884700,1648.392700,2302.365800,2523.560200,2597.782990,2761.135720,...,2833.860815,802.781409,1879.542349,1053.972349,278.290841,0.001794,293.217361,2.470939,0.240179,1.000000
75%,-9.322609,-14.926473,2103.385354,2156.902557,2565.968373,2921.320072,3351.444404,3735.071878,3802.460633,4015.688122,...,4084.349790,1457.314397,2650.952606,1646.218963,294.491275,0.003582,692.439658,4.868427,0.401444,1.000000
max,14.057914,7.074079,13624.663133,14274.620618,15724.538745,18868.088113,17166.621988,16868.053089,17286.252694,16836.840939,...,19489.963954,13943.846879,13135.748022,11376.935402,305.419884,0.045823,5376.089584,73.877508,0.796425,1.000000


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59624 entries, 0 to 59623
Columns: 217 entries, S2_B2_jan to LABELS
dtypes: float64(217)
memory usage: 98.7 MB


In [31]:
df.dtypes

S2_B2_jan                       float64
S2_B3_jan                       float64
S2_B4_jan                       float64
S2_B5_jan                       float64
S2_B6_jan                       float64
                                 ...   
ERA5_total_precipitation_dec    float64
topo_elevation_dec              float64
topo_slope_dec                  float64
NDVI_dec                        float64
LABELS                          float64
Length: 217, dtype: object

In [32]:
labels = df[["LABELS"]]

In [33]:
labels.value_counts()

LABELS
1.0       40364
0.0       19260
dtype: int64

In [34]:
# There is some imbalance in the data. But not much.

In [35]:
df.isna().sum()

S2_B2_jan                       0
S2_B3_jan                       0
S2_B4_jan                       0
S2_B5_jan                       0
S2_B6_jan                       0
                               ..
ERA5_total_precipitation_dec    0
topo_elevation_dec              0
topo_slope_dec                  0
NDVI_dec                        0
LABELS                          0
Length: 217, dtype: int64

In [36]:
df[["LABELS"]] = df[["LABELS"]].astype(int)
df

,S2_B2_jan,S2_B3_jan,S2_B4_jan,S2_B5_jan,S2_B6_jan,S2_B7_jan,S2_B8_jan,S2_B8A_jan,S2_B9_jan,S2_B11_jan,...,S2_B11_dec,S2_B12_dec,S1_VV_dec,S1_VH_dec,ERA5_temperature_2m_dec,ERA5_total_precipitation_dec,topo_elevation_dec,topo_slope_dec,NDVI_dec,LABELS
0,-14.271277,-21.134172,957.531174,1019.557045,1230.022834,1435.138891,1693.702270,1805.411109,1891.557355,2020.567257,...,2203.025469,795.862978,2747.777894,1688.997611,289.382220,0.000147,1886.384195,0.749163,0.307887,0
1,-12.341429,-14.744978,949.227883,895.845779,987.517322,1350.496916,1930.239804,2048.686546,2375.680146,2259.903290,...,2922.382762,1140.796180,2528.590979,1934.261859,266.578370,0.002876,69.876216,1.385904,0.097779,1
2,-12.317847,-17.026201,1087.616069,1086.401035,1185.755955,1529.674085,1858.981635,1904.667487,2082.954737,2115.713139,...,2548.726966,521.102434,1556.082048,741.212901,276.467873,0.001622,91.279743,2.125908,0.418492,1
3,-15.332478,-20.978203,2987.224569,3163.553102,3412.572434,3908.443371,4044.232162,4231.353770,3682.709329,4518.882323,...,2537.545246,1073.588565,2388.141112,1367.682434,276.101799,0.003720,767.817294,8.427714,0.336528,1
4,-11.739502,-11.374262,691.825857,782.111227,478.448214,976.753128,3189.134129,3813.893119,3861.824527,4174.591554,...,4276.302643,371.861994,2080.641023,692.730434,298.663246,0.001763,61.500923,1.203899,0.595404,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59619,-8.163415,-15.237238,987.976575,1151.249683,1243.495362,1570.346203,2985.256589,3424.652420,3504.617647,3786.160943,...,2846.906155,545.811116,4877.934218,2939.084766,293.583787,0.008626,773.400823,3.750506,0.306910,0
59620,-11.412780,-18.545065,1961.306816,1904.491667,2287.616313,2794.906732,2868.472367,2952.698071,2972.389632,3199.591485,...,7299.094172,2067.865931,6147.303097,4271.877307,266.000834,0.003052,195.505610,1.661317,0.076089,1
59621,-19.757668,-18.974376,1369.005380,1696.975995,2378.074718,2426.659176,2511.191824,2731.803974,2676.593677,2916.115610,...,2695.952941,512.207308,3990.546419,2672.906513,299.309719,0.001287,513.766742,3.598575,0.274659,0
59622,-10.536396,-15.386933,3022.282909,3368.075760,3736.875439,4000.780072,4828.072025,5342.635968,5117.677939,5777.358267,...,4433.778527,902.604485,1769.948892,660.408789,292.725187,0.011576,496.085568,12.537847,0.627107,1


In [38]:
X = df.drop(columns=["LABELS"])
X

,S2_B2_jan,S2_B3_jan,S2_B4_jan,S2_B5_jan,S2_B6_jan,S2_B7_jan,S2_B8_jan,S2_B8A_jan,S2_B9_jan,S2_B11_jan,...,S2_B9_dec,S2_B11_dec,S2_B12_dec,S1_VV_dec,S1_VH_dec,ERA5_temperature_2m_dec,ERA5_total_precipitation_dec,topo_elevation_dec,topo_slope_dec,NDVI_dec
0,-14.271277,-21.134172,957.531174,1019.557045,1230.022834,1435.138891,1693.702270,1805.411109,1891.557355,2020.567257,...,1939.707676,2203.025469,795.862978,2747.777894,1688.997611,289.382220,0.000147,1886.384195,0.749163,0.307887
1,-12.341429,-14.744978,949.227883,895.845779,987.517322,1350.496916,1930.239804,2048.686546,2375.680146,2259.903290,...,2816.650882,2922.382762,1140.796180,2528.590979,1934.261859,266.578370,0.002876,69.876216,1.385904,0.097779
2,-12.317847,-17.026201,1087.616069,1086.401035,1185.755955,1529.674085,1858.981635,1904.667487,2082.954737,2115.713139,...,2425.676203,2548.726966,521.102434,1556.082048,741.212901,276.467873,0.001622,91.279743,2.125908,0.418492
3,-15.332478,-20.978203,2987.224569,3163.553102,3412.572434,3908.443371,4044.232162,4231.353770,3682.709329,4518.882323,...,2233.688637,2537.545246,1073.588565,2388.141112,1367.682434,276.101799,0.003720,767.817294,8.427714,0.336528
4,-11.739502,-11.374262,691.825857,782.111227,478.448214,976.753128,3189.134129,3813.893119,3861.824527,4174.591554,...,3569.602117,4276.302643,371.861994,2080.641023,692.730434,298.663246,0.001763,61.500923,1.203899,0.595404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59619,-8.163415,-15.237238,987.976575,1151.249683,1243.495362,1570.346203,2985.256589,3424.652420,3504.617647,3786.160943,...,2487.672188,2846.906155,545.811116,4877.934218,2939.084766,293.583787,0.008626,773.400823,3.750506,0.306910
59620,-11.412780,-18.545065,1961.306816,1904.491667,2287.616313,2794.906732,2868.472367,2952.698071,2972.389632,3199.591485,...,6940.383810,7299.094172,2067.865931,6147.303097,4271.877307,266.000834,0.003052,195.505610,1.661317,0.076089
59621,-19.757668,-18.974376,1369.005380,1696.975995,2378.074718,2426.659176,2511.191824,2731.803974,2676.593677,2916.115610,...,2364.680153,2695.952941,512.207308,3990.546419,2672.906513,299.309719,0.001287,513.766742,3.598575,0.274659
59622,-10.536396,-15.386933,3022.282909,3368.075760,3736.875439,4000.780072,4828.072025,5342.635968,5117.677939,5777.358267,...,4080.569025,4433.778527,902.604485,1769.948892,660.408789,292.725187,0.011576,496.085568,12.537847,0.627107


In [39]:
y = df[["LABELS"]]

# Scaling features:

In [40]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42)

In [41]:
scaler = preprocessing.StandardScaler()

In [42]:
X_train_scaled = scaler.fit_transform(X_train)

In [43]:
X_test_scaled = scaler.transform(X_test)

In [44]:
X_train_scaled_df = pd.DataFrame(X_train_scaled)
X_train_scaled_df

,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
0,-0.797729,0.467761,-0.701365,-0.702984,-0.748642,-0.725264,-0.769341,-0.779900,-0.793630,-0.754435,...,-0.118022,-0.010031,-0.588192,0.561350,-0.106558,0.947571,0.792298,-0.746424,-0.758132,1.279411
1,0.047049,1.363623,0.879251,0.897829,0.742642,0.704733,0.756250,0.822013,0.881316,0.897081,...,-0.002212,0.179060,-0.276631,0.438898,0.476199,0.953089,1.444557,0.166986,-0.029186,-0.029663
2,0.267683,0.373758,-0.354308,-0.389849,-0.342097,-0.406442,-0.645305,-0.712392,-0.688281,-0.710290,...,-0.716206,-0.741774,-0.556300,0.608868,0.802224,1.200449,-0.851614,-0.435071,-0.553942,-0.530480
3,0.887387,-0.512847,-0.582974,-0.586721,-0.513031,-0.585133,-0.837134,-0.939279,-0.952186,-1.014626,...,-0.987101,-1.080562,-0.597187,-0.537060,-0.372377,-0.283541,-0.000803,-0.126894,-0.418413,-0.668741
4,-0.738945,-0.341730,-0.257399,-0.084465,0.186875,0.071321,-0.057264,-0.073353,-0.105668,-0.056955,...,-0.716206,-0.750145,-0.790175,-0.597372,-0.694994,1.073707,0.597244,-0.161842,-0.306274,-0.225121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50675,-2.192558,-3.880796,-0.427825,-0.322675,-0.228449,-0.185430,-0.302825,-0.323588,-0.365848,-0.356617,...,-0.252966,-0.224236,0.087265,0.411483,1.646256,-0.421670,-0.655905,-0.462076,0.180998,-0.853304
50676,-0.717396,-0.261645,-0.594909,-0.606356,-0.714918,-0.574788,-0.155187,-0.113959,-0.033307,-0.041375,...,-0.011275,-0.073062,0.056191,-0.420091,-0.742706,-0.333196,-0.370092,-0.643169,-0.615142,1.377141
50677,-0.305205,-1.050890,0.364632,0.465331,0.629456,0.643602,0.538811,0.523558,0.384898,0.580800,...,0.427293,0.366673,0.258173,1.925680,1.265703,1.092842,-0.421220,-0.485904,-0.097074,-0.016485
50678,1.323620,0.425814,-0.459809,-0.455990,-0.379979,-0.471335,-0.736198,-0.861620,-0.842580,-0.939321,...,-1.000697,-1.137683,-0.438545,-0.781049,-0.633652,-0.239808,-0.164171,-0.392180,-0.418553,-0.816261


In [118]:
data = pd.concat([X_train_scaled_df, labels], axis = 1)

In [119]:
data

,S2_B2_jan,S2_B3_jan,S2_B4_jan,S2_B5_jan,S2_B6_jan,S2_B7_jan,S2_B8_jan,S2_B8A_jan,S2_B9_jan,S2_B11_jan,...,S2_B11_dec,S2_B12_dec,S1_VV_dec,S1_VH_dec,ERA5_temperature_2m_dec,ERA5_total_precipitation_dec,topo_elevation_dec,topo_slope_dec,NDVI_dec,LABELS
0,-0.588311,-0.647427,-0.580671,-0.559091,-0.515052,-0.526498,-0.667178,-0.702364,-0.689416,-0.686334,...,-0.607527,-0.410114,0.580734,0.469020,0.598577,-0.835666,2.361267,-0.615032,0.244305,0
1,-0.129798,0.706188,-0.584963,-0.623077,-0.631322,-0.562673,-0.547283,-0.575619,-0.438097,-0.559792,...,-0.227862,-0.125442,0.392909,0.761239,-1.028275,-0.074048,-0.737733,-0.515400,-0.823474,1
2,-0.124195,0.222888,-0.513439,-0.524517,-0.536276,-0.486095,-0.583402,-0.650652,-0.590057,-0.636029,...,-0.425071,-0.636872,-0.440447,-0.660216,-0.322747,-0.424132,-0.701219,-0.399611,0.806407,1
3,-0.840442,-0.614384,0.468344,0.549836,0.531383,0.530571,0.524252,0.561545,0.240411,0.634576,...,-0.430973,-0.180908,0.272556,0.086190,-0.348863,0.161473,0.452969,0.586443,0.389863,1
4,0.013214,1.420309,-0.717997,-0.681903,-0.875398,-0.722408,0.090822,0.344049,0.333393,0.452542,...,0.486715,-0.760039,0.009055,-0.717980,1.260696,-0.384757,-0.752022,-0.543879,1.705482,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59619,0.862857,0.601898,-0.564936,-0.490976,-0.508592,-0.468712,-0.012519,0.141256,0.147960,0.247171,...,-0.267697,-0.616480,2.406095,1.958433,0.898322,1.530837,0.462494,-0.145407,0.239342,0
59620,0.090840,-0.098898,-0.061886,-0.101381,-0.007983,0.054655,-0.071714,-0.104631,-0.128332,-0.062960,...,2.082095,0.639663,3.493835,3.546385,-1.069477,-0.024856,-0.523407,-0.472306,-0.933700,1
59621,-1.891824,-0.189853,-0.368007,-0.208713,0.035388,-0.102731,-0.252811,-0.219716,-0.281886,-0.212840,...,-0.347368,-0.644213,1.645680,1.641296,1.306816,-0.517642,0.019553,-0.169180,0.075440,0
59622,0.299060,0.570184,0.486463,0.655620,0.686871,0.570035,0.921562,1.140519,0.985335,1.299958,...,0.569828,-0.322020,-0.257182,-0.756490,0.837068,2.354229,-0.010611,1.229562,1.866595,1


In [120]:
#define all the parameters
N_THREADS = 8 # threads cnt for lgbm and linear models
N_FOLDS = 5 # folds cnt for AutoML
RANDOM_STATE = 42 # fixed random state for various reasons
TEST_SIZE = 0.15 # Test size for metric check
TARGET_NAME = 'LABELS' # Target column name

In [121]:
train_data, test_data = train_test_split(data, 
                                         test_size=TEST_SIZE, 
                                         stratify=data[TARGET_NAME],                                          
                                         random_state=RANDOM_STATE)

In [122]:
#Referred to: https://towardsdatascience.com/lightautoml-preset-usage-tutorial-2cce7da6f936
def f1(y_true, y_pred):
    return f1_score(y_true, (y_pred > 0.5).astype(int))

In [123]:
#create task for binary classification
task = Task('binary', metric=f1)

In [124]:
#AutoML can do this automatically for all other variables except for target variables.
roles = {'target': TARGET_NAME,
         }

In [125]:
TIMEOUT = 2700
automl = TabularUtilizedAutoML(task = task, 
                       timeout = TIMEOUT,
                       general_params = {'nested_cv': False, 'use_algos': [['lgb', 'lgb_tuned', 'catb', 'catb_tuned']]},
                       reader_params = {'cv': N_FOLDS, 'random_state': RANDOM_STATE},
                       tuning_params = {'max_tuning_iter': 20, 'max_tuning_time': 75},
                       lgb_params = {'default_params': {'num_threads': N_THREADS}})
oof_pred = automl.fit_predict(train_data, roles = roles, verbose = 1)

[16:11:32] Start automl utilizator with listed constraints:
[16:11:32] - time: 2700.00 seconds
[16:11:32] - CPU: 4 cores
[16:11:32] - memory: 16 GB

[16:11:32] If one preset completes earlier, next preset configuration will be started

[16:11:32] ==================================================
[16:11:32] Start 0 automl preset configuration:
[16:11:32] C:\Users\kaush\anaconda3\envs\ift6758-conda-env\lib\site-packages\lightautoml\automl\presets\tabular_configs\conf_0_sel_type_0.yml, random state: {'reader_params': {'random_state': 42}, 'general_params': {'return_all_predictions': False}}
[16:11:32] Stdout logging level is INFO.
[16:11:32] Task: binary

[16:11:32] Start automl preset with listed constraints:
[16:11:32] - time: 2700.00 seconds
[16:11:32] - CPU: 4 cores
[16:11:32] - memory: 16 GB

[16:11:32] Train data shape: (50680, 217)

[16:11:42] Layer 1 train process start. Time left 2689.53 secs
[16:11:47] Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...
[16:15:54] Fitting Lvl_0_Pipe_

In [126]:
valid_pred = automl.predict(test_data)

In [127]:
print(f'OOF score: {f1(train_data[TARGET_NAME].values, oof_pred.data[:, 0])}')
print(f'HOLDOUT score: {f1(test_data[TARGET_NAME].values, valid_pred.data[:, 0])}')

OOF score: 0.895423150775227
HOLDOUT score: 0.898440038999025


In [128]:
with open('./models/LAMA_automl_utilized_model_4.pkl', 'wb') as files:
    pickle.dump(automl, files)

In [1]:
with open('./models/LAMA_automl_utilized_model_4.pkl', 'rb') as files:
    model = pickle.load(files)

NameError: name 'pickle' is not defined

In [130]:
test = pd.read_csv(r"C:\Users\kaush\Documents\6390_Kaggle_Project_2\test_nolabels.csv")

# Check the test dataframe before you wreck the test dataframe:

In [131]:
test

,S.No,S2_B2_jan,S2_B3_jan,S2_B4_jan,S2_B5_jan,S2_B6_jan,S2_B7_jan,S2_B8_jan,S2_B8A_jan,S2_B9_jan,...,S2_B9_dec,S2_B11_dec,S2_B12_dec,S1_VV_dec,S1_VH_dec,ERA5_temperature_2m_dec,ERA5_total_precipitation_dec,topo_elevation_dec,topo_slope_dec,NDVI_dec
0,0,-17.649477,-24.327178,1111.603354,1190.158872,1361.861147,1436.238138,1350.277481,1382.111848,1419.716749,...,777.782928,966.513644,467.731680,1045.355257,633.790963,296.952437,0.000092,276.466775,0.765980,0.000600
1,1,-14.559686,-19.936119,1764.795592,1856.603435,2156.740323,2633.317506,2731.894209,2878.742338,2621.323845,...,2026.702042,2292.479225,791.909589,2902.059945,1728.924349,298.468523,0.000139,187.130317,2.598537,0.217297
2,2,-14.208844,-21.404320,1210.320260,1246.028476,1523.531488,1706.652761,2014.364032,2198.544215,2277.422666,...,2103.697056,2369.819453,893.709359,3217.008133,2158.612101,298.045025,0.000104,324.857357,0.758805,0.230539
3,3,-13.895412,-21.169488,1550.755198,1541.339240,1732.355678,1828.669115,1876.796177,2001.004560,1996.979443,...,2101.697185,2372.614882,841.326953,3259.568699,1979.892417,298.534272,0.000130,184.338553,1.132985,0.214938
4,4,-11.562658,-13.594914,1171.571568,1143.268311,1296.423152,1376.878830,1488.835033,1530.996415,1667.407479,...,1926.708518,2116.367142,849.233731,2550.935276,1457.042276,298.389238,0.000113,261.577366,0.759891,0.220560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,1195,-14.947773,-18.822142,1190.023326,1293.916708,1634.198686,1820.974390,1991.601005,2146.969823,2272.305089,...,1683.724255,1898.323610,533.950948,2490.286470,1612.946682,298.224866,0.000106,345.330295,2.723475,0.179802
1196,1196,-9.064372,-13.551807,1108.835591,1126.307895,1180.944337,1505.490663,2007.436154,2167.404959,2347.021714,...,1875.711821,2080.958363,948.068460,2299.827937,1361.027977,297.658268,0.000092,235.520899,2.572516,0.257082
1197,1197,-6.723285,-12.445667,1144.816519,1191.156543,1416.713584,1530.773331,1870.857996,1974.730813,1950.921249,...,1501.736041,1685.870938,424.244400,2135.969758,1455.141003,297.323723,0.000092,338.816178,8.861480,0.137137
1198,1198,-9.844837,-14.439588,1027.647855,1018.559373,1012.537732,1283.442883,2019.312516,2181.028384,2323.480860,...,1760.719269,2116.367142,571.508145,1796.549245,835.325927,298.439516,0.000121,248.549132,7.234033,0.297259


In [132]:
test = test.drop(columns=["S.No"])

In [133]:
test=(test-test.mean())/test.std()

In [134]:
test

,S2_B2_jan,S2_B3_jan,S2_B4_jan,S2_B5_jan,S2_B6_jan,S2_B7_jan,S2_B8_jan,S2_B8A_jan,S2_B9_jan,S2_B11_jan,...,S2_B9_dec,S2_B11_dec,S2_B12_dec,S1_VV_dec,S1_VH_dec,ERA5_temperature_2m_dec,ERA5_total_precipitation_dec,topo_elevation_dec,topo_slope_dec,NDVI_dec
0,-1.525466,-1.206610,-0.609914,-0.461846,-0.527089,-0.732780,-1.770525,-1.931417,-1.843035,-2.192362,...,-3.504485,-3.605774,-0.971879,-2.597297,-2.066475,-1.517697,-0.530480,0.528054,-0.607787,-3.230098
1,-0.593724,-0.188969,2.166729,1.795263,1.229047,1.711117,1.661690,1.498394,0.698642,1.475342,...,-0.173636,-0.144933,0.412996,-0.048411,-0.186743,0.682086,-0.141871,-0.216358,-0.168723,-0.201562
2,-0.487926,-0.529229,-0.190280,-0.272627,-0.169909,-0.180715,-0.120800,-0.060408,-0.028789,0.135606,...,0.031708,0.056929,0.847881,0.383951,0.550791,0.067606,-0.433551,0.931278,-0.609506,-0.016499
3,-0.393409,-0.474806,1.256869,0.727529,0.291449,0.068387,-0.462546,-0.513108,-0.621991,-0.549327,...,0.026375,0.064225,0.624105,0.442378,0.244029,0.777485,-0.215205,-0.239621,-0.519856,-0.234539
4,0.310045,1.280624,-0.354996,-0.620654,-0.671662,-0.853965,-1.426320,-1.590220,-1.319112,-1.523498,...,-0.440317,-0.604595,0.657882,-0.530435,-0.653412,0.567046,-0.355257,0.403986,-0.609246,-0.155965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,-0.710753,0.069199,-0.276560,-0.110440,0.074590,0.052678,-0.177348,-0.178601,-0.039614,-0.115469,...,-1.088353,-1.173701,-0.688993,-0.613694,-0.385812,0.328549,-0.413453,1.101872,-0.138789,-0.725588
1196,1.063416,1.290614,-0.621679,-0.678096,-0.926791,-0.591398,-0.138010,-0.131770,0.118429,-0.187779,...,-0.576325,-0.697014,1.080100,-0.875155,-0.818215,-0.493564,-0.530353,0.186866,-0.174957,0.354468
1197,1.769383,1.546966,-0.468728,-0.458467,-0.405903,-0.539782,-0.477298,-0.573319,-0.719415,-0.756213,...,-1.573713,-1.728213,-1.157655,-1.100100,-0.656676,-0.978975,-0.530480,1.047592,1.331821,-1.321878
1198,0.828063,1.084869,-0.966799,-1.043017,-1.298853,-1.044719,-0.108507,-0.100549,0.068635,0.199881,...,-0.883008,-0.604595,-0.528550,-1.566057,-1.720552,0.639998,-0.295661,0.295426,0.941900,0.915976


In [136]:
LAMA_predictions = model.predict(test)
LAMA_predictions

array([[0.10873257],
       [0.9164033 ],
       [0.91681135],
       ...,
       [0.3564803 ],
       [0.37070912],
       [0.7760452 ]], dtype=float32)

In [137]:
LAMA_predictions = LAMA_predictions.data

In [138]:
LAMA_predictions = LAMA_predictions.ravel()

In [139]:
LAMA_predictions = np.where(LAMA_predictions > 0.5, 1, 0)

In [140]:
LAMA_preds_df = pd.DataFrame(LAMA_predictions)

In [141]:
LAMA_preds_df = LAMA_preds_df.rename(columns={0: "LABELS"})
LAMA_preds_df.index.names = ["S.No"]

In [142]:
LAMA_preds_df.value_counts()

LABELS
1         884
0         316
dtype: int64

In [143]:
LAMA_preds_df.to_csv(r"./predictions_submit/LAMA_predictions_new.csv")